In [7]:
import os
import shutil
import numpy as np

## Extract | Extract data from pickle files

In [4]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [5]:
metadata_path = '../cifar-100-python/meta'
metadata = unpickle(metadata_path)
superclass_dict = dict(list(enumerate(metadata[b'coarse_label_names'])))

In [84]:
data_pre_path = '../cifar-100-python/'

# File paths
data_train_path = data_pre_path + 'train'
data_test_path = data_pre_path + 'test'

# Read dictionary
data_train_dict = unpickle(data_train_path)
data_test_dict = unpickle(data_test_path)

# Get data (change the coarse_labels if you want to use the 100 classes)
data_train = data_train_dict[b'data']
label_train = np.array(data_train_dict[b'coarse_labels'])
data_test = data_test_dict[b'data']
label_test = np.array(data_test_dict[b'coarse_labels'])

## Transform | Transform tensor to be in range [0,1]

In [85]:
def transform_range_data(dataset):
    dataset = dataset / 255
    return dataset

In [86]:
data_train = transform_range_data(data_train)
data_test = transform_range_data(data_test)

In [104]:
data_test

numpy.ndarray

## Transform | Retrieve mean and std of channel data in order to normalize data

In [88]:
def get_statistics_of_data(data, channel_nr):
    channel_data_name = []
    if channel_nr == 1:
        channel_data_name = data[:,0:1024]
    elif channel_nr == 2:
        channel_data_name = data[:,1024:2048]
    elif channel_nr == 3:
        channel_data_name = data[:,2048:]

    mean_of_channel = np.mean(channel_data_name)
    std_of_channel = np.std(channel_data_name)
    return mean_of_channel, std_of_channel

In [89]:
first_mean, first_std = get_statistics_of_data(data_train, 1)
second_mean, second_std = get_statistics_of_data(data_train, 2)
third_mean, third_std = get_statistics_of_data(data_train, 3)

In [93]:
print(first_mean, first_std)
print(second_mean, second_std)
print(third_mean, third_std)

0.5070751592371322 0.26733428587924063
0.4865488733149497 0.25643846291708833
0.44091784336703466 0.27615047132568393


In [ ]:
normalize = transforms.Normalize(mean=[first_mean, second_mean, third_mean],
                                 std=[first_std, second_std, third_std]
                                 )

## Load | Create category folders

In [113]:
superclass_dict

{0: b'aquatic_mammals',
 1: b'fish',
 2: b'flowers',
 3: b'food_containers',
 4: b'fruit_and_vegetables',
 5: b'household_electrical_devices',
 6: b'household_furniture',
 7: b'insects',
 8: b'large_carnivores',
 9: b'large_man-made_outdoor_things',
 10: b'large_natural_outdoor_scenes',
 11: b'large_omnivores_and_herbivores',
 12: b'medium_mammals',
 13: b'non-insect_invertebrates',
 14: b'people',
 15: b'reptiles',
 16: b'small_mammals',
 17: b'trees',
 18: b'vehicles_1',
 19: b'vehicles_2'}

In [109]:
dataset_path = "../cifar-100-data"

In [111]:
def extract_categories(dataset_path):
    for category in superclass_dict:
       train_indices = np.where(label_train == category)[0]
       train_data = data_train[train_indices]

       category_folder = os.path.join(dataset_path, str(category))
       os.makedirs(category_folder, exist_ok=True)
       np.save(os.path.join(category_folder, 'train_images.'), train_data)

In [ ]:
extract_categories(dataset_path)